# XtEHR Logical model processing

* ophalen XtEHR Logical Models
* consistente layout met zibs
* klaar voor een mapping - zie mapping Notebook

## Relevante kern content

NL kernset:
- Behandelaanwijzing

ePS required:
- Medicatie
- Diagnose/ Probleemlijst(actief)
- Verrichtingen(<6mnd, ≥ 6mnd)
- Allergieën en intoleranties + - - 
- Alerts
- Hulpmiddel
- Patiënt

ePS recommended + optional:
- Vaccinations
- Implants
- Treatment restrictions
- Vital signs
- History of illness
- Pregnancy
- Social history
- Functional status
- Results
- Plan of Care
- Encounters

# Imports

In [2]:
import pandas as pd
import requests
import json
import os
from io import StringIO

# Download JSON

* Dit hoeft maar een keer te gebeuren, bij nieuwe mappings download de JSON die nodig is. 
* Kan ook door directe download 'raw' JSON.
* Dit zijn de XtEHR Logical Models

In [3]:
xtehrlist = ['FunctionalStatus']
xtehrbase = 'https://build.fhir.org/ig/Xt-EHR/xt-ehr-common/StructureDefinition-EHDS'
# res = 'AlertFlag'
if (not os.path.exists('xtehr')):
    os.makedirs('xtehr')
for res in xtehrlist:
    print(res)
    url = xtehrbase + res + '.json'
    response = requests.get(url)
    jsondata = response.json()
    with open(os.path.join('xtehr', 'downloads', res + '.json'), 'w') as jsonfile:
        json.dump(jsondata, jsonfile, indent=4)

FunctionalStatus


# Kies en lees het te verwerken Logical Model

* Toon en verifieer dat de uitvoer er is

In [4]:
base = 'xtehr'
file = 'FunctionalStatus'
url = os.path.join(base, 'downloads', file + '.json')
with open(url) as jsonfile:
    xlm = json.load(jsonfile)['snapshot']['element']
dfx = pd.DataFrame(xlm)
dfx

,id,path,short,definition,min,max,base,isModifier,mapping,type,binding,requirements,comment
0,EHDSFunctionalStatus,EHDSFunctionalStatus,Functional status,EHDS refined base model for Functional status,0,*,"{'path': 'Base', 'min': 0, 'max': '*'}",False,"[{'identity': 'rim', 'map': 'n/a'}]",NaN,NaN,NaN,NaN
1,EHDSFunctionalStatus.header,EHDSFunctionalStatus.header,Common header for all patient-related data,Common header for all patient-related data,1,1,"{'path': 'EHDSDataSet.header', 'min': 1, 'max'...",NaN,NaN,[{'code': 'Base'}],NaN,NaN,NaN
2,EHDSFunctionalStatus.header.subject,EHDSFunctionalStatus.header.subject,Subject,Patient/subject information,1,1,"{'path': 'EHDSDataSet.header.subject', 'min': ...",NaN,NaN,[{'code': 'https://www.xt-ehr.eu/specification...,NaN,NaN,NaN
3,EHDSFunctionalStatus.header.identifier,EHDSFunctionalStatus.header.identifier,Business identifier for the object,Business identifier for the object,0,*,"{'path': 'EHDSDataSet.header.identifier', 'min...",NaN,NaN,[{'code': 'Identifier'}],NaN,NaN,NaN
4,EHDSFunctionalStatus.header.authorship,EHDSFunctionalStatus.header.authorship,Authorship,Resource authoring details,1,*,"{'path': 'EHDSDataSet.header.authorship', 'min...",NaN,NaN,[{'code': 'Base'}],NaN,NaN,NaN
5,EHDSFunctionalStatus.header.authorship.author[x],EHDSFunctionalStatus.header.authorship.author[x],Author,Author(s) by whom the resource was/were author...,1,1,{'path': 'EHDSDataSet.header.authorship.author...,NaN,NaN,[{'code': 'https://www.xt-ehr.eu/specification...,NaN,NaN,NaN
6,EHDSFunctionalStatus.header.authorship.datetime,EHDSFunctionalStatus.header.authorship.datetime,Date and time of authoring/issuing,Date and time of the issuing the document/reso...,1,1,{'path': 'EHDSDataSet.header.authorship.dateti...,NaN,NaN,[{'code': 'dateTime'}],NaN,NaN,NaN
7,EHDSFunctionalStatus.header.lastUpdate,EHDSFunctionalStatus.header.lastUpdate,Date and time of the last update to the resource,Date and time of the last update to the docume...,0,1,"{'path': 'EHDSDataSet.header.lastUpdate', 'min...",NaN,NaN,[{'code': 'dateTime'}],NaN,NaN,NaN
8,EHDSFunctionalStatus.header.status,EHDSFunctionalStatus.header.status,Status of the resource,Status of the resource,1,1,"{'path': 'EHDSDataSet.header.status', 'min': 1...",NaN,NaN,[{'code': 'CodeableConcept'}],NaN,NaN,NaN
9,EHDSFunctionalStatus.header.statusReason[x],EHDSFunctionalStatus.header.statusReason[x],Reason for the current status of the resource.,Reason for the current status of the resource.,0,1,"{'path': 'EHDSDataSet.header.statusReason[x]',...",NaN,NaN,"[{'code': 'CodeableConcept'}, {'code': 'string'}]",NaN,NaN,NaN


# Opschonen uitvoer zodat zib en xtehr consistent zijn

* een card. veld met n..m
* type (is een JSON object) omzetten naar tekst
* alleen nodige kolommen houden

In [5]:
dfx.insert(2, 'card.', None)
dfx['card.'] = dfx['min'].astype(str) + '..' + dfx['max'].astype(str)
for i, row in dfx.iterrows():
    if 'type' in row:
        # row['type'][0]['code']
        try:
            dfx.at[i, 'type'] = row['type'][0]['code'].split('/')[-1]
        except TypeError:
            pass
dfx= dfx[['id', 'path', 'short', 'definition', 'type', 'card.', 'binding']]
dfx

,id,path,short,definition,type,card.,binding
0,EHDSFunctionalStatus,EHDSFunctionalStatus,Functional status,EHDS refined base model for Functional status,NaN,0..*,NaN
1,EHDSFunctionalStatus.header,EHDSFunctionalStatus.header,Common header for all patient-related data,Common header for all patient-related data,Base,1..1,NaN
2,EHDSFunctionalStatus.header.subject,EHDSFunctionalStatus.header.subject,Subject,Patient/subject information,EHDSPatient,1..1,NaN
3,EHDSFunctionalStatus.header.identifier,EHDSFunctionalStatus.header.identifier,Business identifier for the object,Business identifier for the object,Identifier,0..*,NaN
4,EHDSFunctionalStatus.header.authorship,EHDSFunctionalStatus.header.authorship,Authorship,Resource authoring details,Base,1..*,NaN
5,EHDSFunctionalStatus.header.authorship.author[x],EHDSFunctionalStatus.header.authorship.author[x],Author,Author(s) by whom the resource was/were author...,EHDSHealthProfessional,1..1,NaN
6,EHDSFunctionalStatus.header.authorship.datetime,EHDSFunctionalStatus.header.authorship.datetime,Date and time of authoring/issuing,Date and time of the issuing the document/reso...,dateTime,1..1,NaN
7,EHDSFunctionalStatus.header.lastUpdate,EHDSFunctionalStatus.header.lastUpdate,Date and time of the last update to the resource,Date and time of the last update to the docume...,dateTime,0..1,NaN
8,EHDSFunctionalStatus.header.status,EHDSFunctionalStatus.header.status,Status of the resource,Status of the resource,CodeableConcept,1..1,NaN
9,EHDSFunctionalStatus.header.statusReason[x],EHDSFunctionalStatus.header.statusReason[x],Reason for the current status of the resource.,Reason for the current status of the resource.,CodeableConcept,0..1,NaN


# Opslaan in Excel

In [6]:
url = os.path.join(base, 'mappable', file + '.xlsx')
dfx.to_excel(url, index=False)